In [14]:
#Reading the xlsx document
import pandas as pd
import numpy as np
import plotly as py
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from math import floor
init_notebook_mode(connected=True)

arq = openpyxl.load_workbook('planilhas.xlsx') 
matricula_aluno = int(67106); #matricula do aluno
#Estrutura do arquivo xlsx
#Geral: Identificação | Professor | Período | Peso | Matéria
#'Nome_d_matéria': Matrícula | Nome | Avaliação 1 | Peso A1 | Avaliação 2 | Peso A2...

#Estrutura do Data_base: { //Dicionário
# 'Nome' : DANIEL JACOB TONN OK 
# 'Matricula' : 2019000000   OK
# 'Estatística': { //Dicionário
#                   'Período': 2022.2
#                   'Peso': 2       
#                   'Nota': (7.0) //Tupla
#                   'Avaliação1': (7.0, 2) //Tupla
#                   'Avaliação2': (8.0, 2) //Tupla
#                          }
# 'Análise Real': { //Dicionário
#                   'Período': 2022.2
#                   'Peso': 2       
#                   'Nota': (7.0) //Tupla
#                   'Avaliação1': (7.0, 2) //Tupla
#                   'Avaliação2': (8.0, 2) //Tupla
#                          }
# }


#Estrutura do CR: { //Dicionário
# 'Acumulado': 7.0
# 'Período2022.2': 7.0
# 'Período2022.1': 7.0
# 'Período2021.2': 7.0
# }


def build(matricula, arq ): #function to find the subjects of the student
    Data_base = {}; #dicionario com as informações
    Data_base['Matrícula'] = matricula_aluno; 
    geral = pd.read_excel('planilhas.xlsx', sheet_name=0)#sheet with the general information
    for i in range(1, len(arq.sheetnames)): #for each sheet
        nome_materia = '';#nome das matérias 
        nota_materia = 0;# nota das matérias 
        peso_materia = 0; # peso das matérias
        nota_por_avaliacao = []; #lista com as notas das avaliações
        dados_materia = {}; #dicionario com as informações da matéria
        planilha = pd.read_excel('planilhas.xlsx', sheet_name=i) #read the sheet
        for j in range(0, len(planilha)): #for each line in sheet
            if planilha.iloc[j, 0] == matricula: #if the student is in the sheet
                Data_base['Nome'] = planilha.iloc[j, 1] #add the name to the dictionary
                nota = 0;
                for k in range(0, len(geral)): #for each line in the general sheet
                    if (geral.iloc[k, 0] == arq.sheetnames[i]): #find the name of the subject
                        nome_materia = geral.iloc[k]['Matéria']; #add the name of the subject to the list
                        dados_materia['Período'] = geral.iloc[k]['Período']; #add the period to the dictionary
                        dados_materia['Peso'] = geral.iloc[k]['Peso'] #add the weight of the subject to the list
                        break; #break for k
                soma_pesos = 0; #sum of the weights
                for k in range(2, len(planilha.columns)): #for each column in the sheet
                    if(k%2 == 0):
                        soma_pesos += planilha.iloc[j, k+1]; #add the weight to the sum
                        nota_por_avaliacao.append((planilha.iloc[j, k], planilha.iloc[j, k+1])) #add the grade and the weight of the grade to the list
                        nota += planilha.iloc[j, k]*planilha.iloc[j, k+1] #add the grade to the total grade
                nota = nota/soma_pesos; #calculate the final grade
                dados_materia['Nota'] = nota; #add the final grade to the dictionary
                for k in range(0, len(nota_por_avaliacao)):
                    dados_materia['Avaliação'+str(k+1)] = nota_por_avaliacao[k];
                break; #break for j
        Data_base[nome_materia] = dados_materia; #add the dictionary to the dictionary
    return Data_base; #return the dictionary

def calculateCR(Data_base): #function to calculate the CR
    soma_pesos = 0; #sum of the weights
    soma_notas = 0; #sum of the grades
    CR = {}
    periodos_cursados = [];
    for i in Data_base: #for each subject
        if(i != 'Nome' and i != 'Matrícula'): #if the subject is not the name or the registration number
            soma_pesos += Data_base[i]['Peso']; #add the weight to the sum
            soma_notas += Data_base[i]['Peso']*Data_base[i]['Nota']; #add the grade to the sum
            if Data_base[i]['Período'] not in periodos_cursados:
                periodos_cursados.append(Data_base[i]['Período']);
    CR['Acumulado'] = soma_notas/soma_pesos; #calculate the CR
    for i in periodos_cursados: #for each subject
        soma_pesos = 0; #sum of the weights
        soma_notas = 0; #sum of the grades
        for j in Data_base: #for each subject
            if(j!= 'Nome' and j != 'Matrícula' and Data_base[j]['Período'] == i):
                soma_pesos += Data_base[j]['Peso'];
                soma_notas += Data_base[j]['Peso']*Data_base[j]['Nota'];
        CR['Período'+str(i)] = soma_notas/soma_pesos;
    return CR; #return the CR

def plot(Date_base):
    #preciso de uma função que plote o gráfico
    CR = calculateCR(Date_base);
    #encontra quais são os períodos
    periodos = [];
    #periodo: [2021, 2022, 2023, 2024]
    for i in Date_base:
        if(i != 'Nome' and i != 'Matrícula'):
            if Date_base[i]['Período'] not in periodos:
                periodos.append(Date_base[i]['Período']);
    
    #separa as matérias por período
    materias = [];
    #materias = [[AL,MG], [KL]]
    for i in periodos: #para cada período
        materias.append([]);#adiciono uma lista vazia no período
    for i in Date_base: #para cada matéria
        if(i != 'Nome' and i != 'Matrícula'):
            for j in range(0, len(periodos)):
                if(Date_base[i]['Período'] == periodos[j]):
                    materias[j].append(i);
    #separa as notas por período
    notas = [];
    #notas = [[9,8], [7]]
    for i in periodos: #para cada período
        notas.append([]);#adiciono uma lista vazia no período
    for i in Date_base: #para cada matéria
        if(i != 'Nome' and i != 'Matrícula'):
            for j in range(0, len(periodos)):
                if(Date_base[i]['Período'] == periodos[j]):
                    notas[j].append(Date_base[i]['Nota']);
    #separa os pesos por período
    pesos = []; #acumulativo
    #pesos = [[2,3], [1]]
    for i in periodos: #para cada período
        pesos.append([]);#adiciono uma lista vazia no período
    for i in Date_base: #para cada matéria
        if(i != 'Nome' and i != 'Matrícula'):
            for j in range(0, len(periodos)):
                if(Date_base[i]['Período'] == periodos[j]):
                    pesos[j].append(Date_base[i]['Peso']);
    for i in range(0,len(pesos)):
        soma_pesos_materias = 0; #zero pra cada período
        #include 0 in the first position
        for j in range(0,len(pesos[i])):
            soma_pesos_materias += pesos[i][j]+0.5;
            pesos[i][j] = soma_pesos_materias;
    for i in range(0,len(pesos)):
        pesos[i].insert(0,0);
    #separa as avaliações por período

    #avaliacoes = [[9,8], [7]]
    # for i in periodos: #para cada período
    #     nota_avaliacoes.append([]);#adiciono uma lista vazia no período
    # for i in Date_base: #para cada matéria
    #     if(i != 'Nome' and i != 'Matrícula'):
    #         for j in range(0, len(periodos)):
    #             if(Date_base[i]['Período'] == periodos[j]):
    #                 nota_avaliacoes[j].append(Date_base[i]['Avaliação1'][0]);
    nota_avaliacoes = []; 
    peso_avaliacoes = []; #cumulativo
    #avaliacoes = [[0,4], [0]]
    for i in periodos: #para cada período
        peso_avaliacoes.append([]);#adiciono uma lista vazia no período
        nota_avaliacoes.append([]);
    for i in range(0,len(materias)):
        for j in materias[i]:
                peso_avaliacoes[i].append([]);
                nota_avaliacoes[i].append([]);
    for i in Date_base: #para cada matéria in Data_base
        if(i != 'Nome' and i != 'Matrícula'):
            sumpeso = 0 
            for j in Data_base[i]: #para cada avaliação
                if(j != 'Período' and j != 'Peso' and j != 'Nota'):
                    #tenho a avaliação, falta encontrar onde incluir
                    for k in range(0, len(materias)):
                        for l in range(0, len(materias[k])):
                            if(materias[k][l] == i):
                                sumpeso += Date_base[i][j][1];
                                peso_avaliacoes[k][l].append(sumpeso);
                                nota_avaliacoes[k][l].append(Date_base[i][j][0]);
    for i in range(0, len(peso_avaliacoes)):# para cada período
        for j in range(0, len(peso_avaliacoes[i])): #para cada matéria
            soma_pesos = 0;
            for k in range(0, len(peso_avaliacoes[i][j])):
                soma_pesos += peso_avaliacoes[i][j][k]
            for k in range(0, len(peso_avaliacoes[i][j])):
                peso_avaliacoes[i][j][k] = peso_avaliacoes[i][j][k]*notas[i][j]/soma_pesos;
                #definindo quantidade de linhas a plotar
    for i in range(0,len(peso_avaliacoes)):#pra cada periodo
        for j in range(0,len(peso_avaliacoes[i])):#pra cada materia
            somatorio = 0;
            for k in range(0,len(peso_avaliacoes[i][j])):#pra cada avaliação
                somatorio += peso_avaliacoes[i][j][k];
                peso_avaliacoes[i][j][k] = somatorio;
    linhas = 0;
    linha = 0;
    coluna = 0;
    if len(periodos) % 2 == 0:
        linhas = len(periodos)/2;
    else:
        linhas = (len(periodos)+1)/2;
    linhas = int(linhas);
    #makeake subplots with plotly
    for i in range(0, len(periodos)):
        periodos[i] = str(periodos[i]);
    xmax = 0;
    for i in pesos:
        for j in i:
            if(j > xmax):
                xmax = j;
    xmax+=0.5;
    #PLOTPLOTPLOTPLOTPLOTPLOTPLOTPLOTPLOTPLOT
    fig = make_subplots(rows=linhas, cols=2, subplot_titles=periodos)
    fig.update_yaxes(range=[0, 10])
    #fig_add_bar
    #ERRO AQUI #############################################################################
    for i in range(0, len(periodos)): #pra cada período
        if(i%2 == 0):
            coluna = 1;
            linha +=1;
        else:
            coluna = 2;
        #
        for j in range(0, len(materias[i])): #pra cada matéria
            xminimo = pesos[i][j];
            xmaximo = pesos[i][j+1]
            for k in range(0, len(peso_avaliacoes[i][j])): #pra cada avaliação
                if nota_avaliacoes[i][j][k] <= 6:
                    color = 'red'
                elif nota_avaliacoes[i][j][k]  < 7:
                    color = 'yellow'
                else:
                    color = 'green'
                if(k == 0):
                    ali = (xmaximo+xminimo+0.5)/2
                    fig.add_shape( type = "rect" , x0 = xminimo+0.5 ,  y0 = 0 ,  x1 = xmaximo ,  y1 = peso_avaliacoes[i][j][k],
                     line=dict(color="White"), text = 'Avaliação'+str(k+1)+str(Data_base[materias[i][j]]['Avaliação'+str(k+1)][0])+'%',
                      fillcolor=color, row=linha, col=coluna)
                    fig.add_trace(go.Scatter( x=[ali], y=[0], mode="markers+text", name="Markers and Text", text=[materias[i][j]], textposition="top center"), row=linha, col=coluna)
                        # fig . add_trace ( 
                        #     go . Scatter (  x = [ xminimo ,  xmaximo ],  y = [peso_avaliacoes[i][j][k], 0]), row=linha, col=coluna) 
                        # fig.add_trace(
                        #     go.Bar(x=[xmaximo], y=[peso_avaliacoes[i][j][k]], base=[0]), row=linha, col=coluna)
                else:
                    fig . add_shape( type = "rect" , x0 = xminimo+0.5 ,  y0 = peso_avaliacoes[i][j][k-1] ,  x1 = xmaximo ,  y1 = peso_avaliacoes[i][j][k], line=dict(color="White"),fillcolor=color, row=linha, col=coluna)
                        # fig.add_trace(
                        #     go.Bar(x=[xmaximo], y=[peso_avaliacoes[i][j][k]], base=[peso_avaliacoes[i][j][k-1]]), row=linha, col=coluna)
                        #  fig . add_trace ( 
                        #     go . Scatter (  x = [ xminimo ,  xmaximo ],  y = [peso_avaliacoes[i][j][k], peso_avaliacoes[i][j][k-1]]), row=linha, col=coluna) 
                            # fig.add_trace(
                            # go.Bar(x=[pesos[i][j-1], pesos[i][j]], y=[peso_avaliacoes[i][j][k]],base=[peso_avaliacoes[i][j][k-1]]), row=linha, col=coluna)
        fig.add_shape( type = "line" , x0 = 0 ,  y0 = round(CR['Período'+str(periodos[i])],2) ,
        x1 = xmax ,  y1 = round(CR['Período'+str(periodos[i])],2), line=dict(color="blue"),
        row=linha, col=coluna)
        fig.add_shape( type = "line" , x0 = 0 ,  y0 = round(CR['Acumulado'],2) ,
        x1 = xmax ,  y1 = round(CR['Acumulado'],2), line=dict(color="lightgreen"),
        row=linha, col=coluna)
    #ERRO AQUI #############################################################################
        # if len(materias[i]) > xmax:
        #     xmax = len(materias[i]);
        #give a periodo as title for each subplot
        # fig.update_xaxes(range=[0, xmax], row=linha, col=coluna)
    # fig.update_layout(title_text=Data_base["Nome"]+'-'+str(Data_base['Matrícula']), title_x=0.5);
    #show the plotly graph in the jupyter notebook
    # iplot(fig);
    # save the html
    fig.write_html("test.html", auto_open=True)

#chamar função build
Data_base = build(matricula_aluno, arq);
plot(Data_base);


ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Shape: 'text'

Did you mean "type"?

    Valid properties:
        editable
            Determines whether the shape could be activated for
            edit or not. Has no effect when the older editable
            shapes mode is enabled via `config.editable` or
            `config.edits.shapePosition`.
        fillcolor
            Sets the color filling the shape's interior. Only
            applies to closed shapes.
        fillrule
            Determines which regions of complex paths constitute
            the interior. For more info please visit
            https://developer.mozilla.org/en-
            US/docs/Web/SVG/Attribute/fill-rule
        layer
            Specifies whether shapes are drawn below or above
            traces.
        line
            :class:`plotly.graph_objects.layout.shape.Line`
            instance or dict with compatible properties
        name
            When used in a 